In [39]:
import tabula as tb
import pandas as pd
import PyPDF2
from PyPDF2 import PdfReader
import re
import docx
from docx import Document
import zipfile
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import os
import pandas as pd
import csv
import json
import re

In [40]:
def filterHTMLstr(str):
    html_tag = {'&#xA;': '\n', '&quot;': '\"', '&amp;': '', '&lt;': '<', '&gt;': '>',
                '&apos;': "'", '&nbsp;': ' ', '&yen;': '¥', '&copy;': '©', '&divide;': '÷'
        , '&times;': 'x', '&trade;': '™', '&reg;': '®', '&sect;': '§', '&euro;': '€',
                '&pound;': '£', '&cent;': '￠', '&raquo;': '»', '&nbsp': ' ', u'\xa0': ' ',
                '\n': ' ', '\t': ' ', '    ': '', '&emsp': ' ', ' ,':',', '  ':' '
                }
    for k, v in html_tag.items():
        str = str.replace(k, v)
        # str = str.replace(k[1:], v)
    str = " ".join(str.split())
    str = str.strip('\n')
    str = str.strip(' ')
    return str

def MultipleFileProcess(files_position):
    totaldata = []
    for position in files_position:
        data = SingleHTMLProcess(position)
        totaldata.append(data)

    return totaldata

def to_CSV(totaldata):
    pd.DataFrame(totaldata).to_csv('./result/2018.csv', index=False, encoding="utf-8")

In [41]:
### code entry
path = "./data/2018.html"
htmlfile = open(path, 'r', encoding='utf-8')
html=htmlfile  
bs = BeautifulSoup(html, "lxml")  # 缩进格式
totaldata = handle(bs)
to_CSV(totaldata)

topic name: Focus Area 1 In-Space Propulsion Technologies
subtopic name: Z10.01Cryogenic Fluid Management
subtopic name: Z10.02Propulsion Systems for Robotic Science Missions
subtopic name: Z10.03Nuclear Thermal Propulsion (NTP)
topic name: Focus Area 2 Power, Energy and Storage
subtopic name: S3.01Power Generation and Conversion
subtopic name: S3.03Power Electronics and Management, and Energy Storage
subtopic name: T3.02Intelligent/Autonomous Electrical Power Systems
subtopic name: T3.03Bio-inspired Concepts for the Development of Power, Energy and Storage Technologies for Air and Space
subtopic name: Z1.01High Power Arrays for Solar Electric Propulsion
subtopic name: Z1.03Fission Surface Power Generation
topic name: Focus Area 3 Autonomous Systems for Space Exploration
subtopic name: H6.02Resilient Autonomous Systems
subtopic name: H6.03Spacecraft Autonomous Agent Cognitive Architectures for Human Exploration
subtopic name: S5.05Fault Management Technologies
subtopic name: T4.03Coord

In [42]:
def handle(bs):
    totalData = []
    ### get overall topic
    evens = bs.find_all("li", {"class": "md-taxonomy even"})
    odds = bs.find_all("li", {"class": "md-taxonomy odd"})
    num_dic = {}
    for index in range(22):
        dic = {}
        topic_block = evens[int(index / 2)] if int(index % 2) == 0 else odds[int(index / 2)]
        dic["Focus Area"] = getTopicName(topic_block)
        ### get sub-focus topics
        subtopic_even = topic_block.find_all("li", {"class": "first-topic even"})
        subtopic_odd = topic_block.find_all("li", {"class": "first-topic odd"})
        for index2 in range(len(subtopic_even) + len(subtopic_odd)):
            subtopic_block = subtopic_even[int(index2 / 2)] if int(index2 % 2) == 0 else subtopic_odd[int(index2 / 2)]
            dic["Subtopic"] = getSubtopicName(subtopic_block)
            metaDic = getMetaData(subtopic_block)
            for k in metaDic:
                # if k in dic.keys():
                dic[k] = metaDic[k]
            ### get other data
            whole_info_block = subtopic_block.find("div", {"class": "whole-topic-content"})
            contents = whole_info_block.contents
            idx = 0
            idxMax = len(contents) - 1
            title = ""
            content = ""
            new_dic = dic.copy()
            new_dic["Content"] = filterHTMLstr(whole_info_block.get_text())
            # print(new_dic)
            while idx < idxMax:
                i = contents[idx]
                if i.find("strong") != None and i.find("strong") != -1 and filterHTMLstr(i.find("strong").get_text()).strip() != "" and filterHTMLstr(i.find("strong").get_text()).strip()[0].isupper():
                    idx += 1
                    if filterHTMLstr(i.get_text()) == "":
                        continue
                    title_raw = i.find("strong")
                    title = filterHTMLstr(title_raw.get_text()).replace(":", "").strip()
                    if title_raw.next_sibling != None:
                        for j in title_raw.next_siblings:
                            tmp = filterHTMLstr(j.get_text()).strip()
                            if tmp != "":
                                content += tmp + '\n'
                        if content != "":
                            content = content.replace(":", "").strip()
                            if new_dic.get(title) != None:
                                totalData.append(new_dic)
                                # print(new_dic)
                                # print()
                                new_dic = dic.copy()
                            if title != "" and content.strip() != "":
                                new_dic[title] = content
                            new_dic[title] = content
                            title = ""
                            content = ""
                            # print(new_dic)
                else:
                    while (idx < idxMax) and ((i.find("strong") == None) or (i.find("strong") == -1) or filterHTMLstr(i.find("strong").get_text()).strip() == "" or not filterHTMLstr(i.find("strong").get_text()).strip()[0].isupper()):
                        if filterHTMLstr(i.get_text()).strip() != "":
                            content += (filterHTMLstr(i.get_text()) + '\n')
                        idx += 1
                        i = contents[idx]
                    if new_dic.get(title) != None:
                        totalData.append(new_dic)
                        # print(new_dic)
                        # print()
                        new_dic = dic.copy()
                    if title != "" and content.strip() != "":
                        new_dic[title] = content
                    title = ""
                    content = ""
            totalData.append(new_dic)
    return totalData

In [43]:
### get topic and subtopic name
def getTopicName(topic_block):
    topic_name = filterHTMLstr(topic_block.find("h2").get_text())
    print("topic name:", topic_name)  # topic name
    return topic_name

### handle every sub-topic
def getSubtopicName(subtopic_block):
    subtopic_name = filterHTMLstr(subtopic_block.find("p", {"class": "subtopic"}).get_text())
    print("subtopic name:", subtopic_name)  # subtopic name
    return subtopic_name

In [44]:
### get Lead Center, Participating Center(s), Solicitation Year
def getMetaData(subtopic_block):
    dic = {}
    info_group = subtopic_block.find_all("p", {"class": "l-center"})
    for raw_info in info_group:
        info = filterHTMLstr(raw_info.get_text())
        pair = info.split(":")
        dic[pair[0]] = pair[1]
        # print(pair[0], pair[1])
    # print(dic)
    return dic

In [45]:
dic = {}
topic_block = bs.find_all("li", {"class": "md-taxonomy even"})[0]
dic["Focus Area"] = getTopicName(topic_block)
subtopic_block = topic_block.find_all("li", {"class": "first-topic even"})[0]
dic["Subtopic"] = getSubtopicName(subtopic_block)[0]
metaDic = getMetaData(subtopic_block)
whole_info_block = subtopic_block.find("div", {"class": "whole-topic-content"})
# filterHTMLstr(whole_info_block.get_text())

        
contents = whole_info_block.contents
idx = 0
idxMax = len(contents) - 1
title = ""
content = ""
while idx < idxMax:
    # print("title")
    i = contents[idx]
    if i.find("strong") != None:
        idx += 1
        if filterHTMLstr(i.get_text()) == "":
            continue
        title_raw = i.find("strong")
        title = filterHTMLstr(title_raw.get_text())
        print(filterHTMLstr(title_raw.get_text()))
        if title_raw.next_sibling != None:
            content = filterHTMLstr(title_raw.next_sibling.get_text()).strip()
            if content != "":
                dic[title] = content
                content = content.replace(":", "").strip()
                print(filterHTMLstr(content))
                title = ""
                content = ""
    else:
        # print("content")
        while (idx < idxMax) and ((i.find("strong") == None) or (i.find("strong") == -1)):
            # print("pre: ", content, i, i.find("strong"), (i.find("strong") == None) or (i.find("strong") == -1))
            if filterHTMLstr(i.get_text()).strip() != "":
                print(filterHTMLstr(i.get_text()))
                content += (filterHTMLstr(i.get_text()) + '\n')
            idx += 1
            i = contents[idx]
        
        print()
        if title != "":
            dic[title] = content
        if title == "References":
            print(dic)
            # break
            print("===================")
        title = ""
        content = ""

topic name: Focus Area 1 In-Space Propulsion Technologies
subtopic name: Z10.01Cryogenic Fluid Management
This subtopic solicits technologies related to cryogenic propellants (such as hydrogen, oxygen, and methane) storage, and transfer to support NASA's exploration goals. This includes a wide range of applications, scales, and environments consistent with future NASA missions. Such missions include but are not limited to a Methane Upper Stage, Nuclear Thermal Propulsion, Lander Propulsion, and In-Situ Resource Utilization in support of the Evolvable Mars Campaign.
Specifically, listed in order of importance:
Develop reliable cryogenic screen channel acquisition devices (NASA is mainly interested in screens with pore sizes < 100 µm) using innovative manufacturing techniques to minimize stresses of cryogenic screen channels to improve screen-to-window manufacturing reliability. Reliability should be based on changes in bubble point pressure before and after thermal cycling the elements 

In [46]:
for i in odds:
    getHigherTopicName(i)

In [47]:
### code entry
path = "./data/2018.html"
htmlfile = open(path, 'r', encoding='utf-8')
html=htmlfile  
bs = BeautifulSoup(html, "lxml")  # 缩进格式
totaldata = handle(bs)
to_CSV(totaldata)

topic name: Focus Area 1 In-Space Propulsion Technologies
subtopic name: Z10.01Cryogenic Fluid Management
subtopic name: Z10.02Propulsion Systems for Robotic Science Missions
subtopic name: Z10.03Nuclear Thermal Propulsion (NTP)
topic name: Focus Area 2 Power, Energy and Storage
subtopic name: S3.01Power Generation and Conversion
subtopic name: S3.03Power Electronics and Management, and Energy Storage
subtopic name: T3.02Intelligent/Autonomous Electrical Power Systems
subtopic name: T3.03Bio-inspired Concepts for the Development of Power, Energy and Storage Technologies for Air and Space
subtopic name: Z1.01High Power Arrays for Solar Electric Propulsion
subtopic name: Z1.03Fission Surface Power Generation
topic name: Focus Area 3 Autonomous Systems for Space Exploration
subtopic name: H6.02Resilient Autonomous Systems
subtopic name: H6.03Spacecraft Autonomous Agent Cognitive Architectures for Human Exploration
subtopic name: S5.05Fault Management Technologies
subtopic name: T4.03Coord